In [1]:
import torch
import torch.optim as optim
import numpy as np
import os
import argparse
import time, datetime
import matplotlib; matplotlib.use('Agg')
from src import config, data
from src.checkpoints import CheckpointIO
from collections import defaultdict
import shutil
from tensorboardX import SummaryWriter

In [2]:
writer = SummaryWriter('testrun')
cfg = config.load_config('configs/pointcloud/grid.yaml', 'configs/default.yaml')
is_cuda = (torch.cuda.is_available())
device = torch.device("cuda" if is_cuda else "cpu")

In [3]:
t0 = time.time()

# Shorthands
out_dir = cfg['training']['out_dir']
batch_size = cfg['training']['batch_size']
backup_every = cfg['training']['backup_every']
vis_n_outputs = cfg['generation']['vis_n_outputs']

In [4]:
# Output directory
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

shutil.copyfile('configs/pointcloud/grid.yaml', os.path.join(out_dir, 'config.yaml'))

# Dataset
train_dataset = config.get_dataset('train', cfg)
val_dataset = config.get_dataset('val', cfg, return_idx=True)

In [5]:
shape = train_dataset[0]

In [6]:
# train_loader = torch.utils.data.DataLoader(
#     train_dataset[5], batch_size=batch_size, num_workers=cfg['training']['n_workers'], shuffle=True,
#     collate_fn=data.collate_remove_none,
#     worker_init_fn=data.worker_init_fn)

# print(len(train_loader))    
# print(train_loader)

# val_loader = torch.utils.data.DataLoader(
#     val_dataset, batch_size=1, num_workers=cfg['training']['n_workers_val'], shuffle=False,
#      collate_fn=data.collate_remove_none,
#      worker_init_fn=data.worker_init_fn)


In [7]:
# code subset
train_dataset = config.get_dataset("train", cfg)
ds = torch.utils.data.Subset(train_dataset, indices= [0]*len(train_dataset))
val_dataset = config.get_dataset("val", cfg)
val_ds = torch.utils.data.Subset(val_dataset, indices= [0]*len(val_dataset))
train_loader = torch.utils.data.DataLoader(
    ds, batch_size=32, num_workers=8, shuffle=True,
    collate_fn=data.collate_remove_none,
    worker_init_fn=data.worker_init_fn)
# val_loader = torch.utils.data.DataLoader(
#     ds, batch_size=1, num_workers=8, shuffle=True,
#     collate_fn=data.collate_remove_none,
#     worker_init_fn=data.worker_init_fn)

In [8]:
# Model
model = config.get_model(cfg, device=device, dataset=train_dataset)

# Generator
generator = config.get_generator(model, cfg, device=device)

# Intialize training
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
trainer = config.get_trainer(model, optimizer, cfg, device=device)

checkpoint_io = CheckpointIO(out_dir, model=model, optimizer=optimizer)
# try:
#     load_dict = checkpoint_io.load('model.pt')
# except FileExistsError:
#     load_dict = dict()
# epoch_it = load_dict.get('epoch_it', 0)
# it = load_dict.get('it', 0)
# metric_val_best = load_dict.get(
#      'loss_val_best', -model_selection_sign * np.inf)

# if metric_val_best == np.inf or metric_val_best == -np.inf:
#     metric_val_best = -model_selection_sign * np.inf
# print('Current best validation metric (%s): %.8f'
#       % (model_selection_metric, metric_val_best))

In [9]:
# Shorthands
print_every = cfg['training']['print_every']
checkpoint_every = cfg['training']['checkpoint_every']
validate_every = cfg['training']['validate_every']
visualize_every = cfg['training']['visualize_every']

# Print model
nparameters = sum(p.numel() for p in model.parameters())
print('Total number of parameters: %d' % nparameters)

print('output path: ', cfg['training']['out_dir'])

Total number of parameters: 1978275
output path:  out/pointcloud/grid


In [10]:
#batch = next(train_loader.__iter__())

In [11]:
# c = model.encode_inputs(batch.get('inputs').cuda())
# results = model.decode(batch.get('points').cuda(), c).logits

In [12]:
it = 0
epoch_it = 0
batch = next(train_loader.__iter__())
while epoch_it <= 30000:
    epoch_it += 1

    it += 1
    loss = trainer.train_step(batch)
    #logger.add_scalar('train/loss', loss, it)
    writer.add_scalars("metropolis+sigma1", {'global_loss':loss[0],
                                        'norm_loss':loss[1],
                                        'gradient_loss':loss[2], 
                                        'ratio_relu':loss[3]}, it)
    
    
    #writer.add_scalar("dotProductReg", loss[0], it)
    # Print output
    if print_every > 0 and (it % print_every) == 0:
        t = datetime.datetime.now()
        print('[Epoch %02d] it=%03d, loss=%.4f, time: %.2fs, %02d:%02d'
                    % (epoch_it, it, loss[0], time.time() - t0, t.hour, t.minute))

    #data_v = next(in data_vis_.__iter__())ist:
    # Visualize output
#     if visualize_every > 0 and (it % visualize_every) == 0:
#         print('Visualizing')
#         for data_vis in data_vis_list:
#             if cfg['generation']['sliding_window']:
#                 out = generator.generate_mesh_sliding(data_vis['data'])
#             else:
#                 out = generator.generate_mesh(data_vis['data'])
#             # Get statistics
#             try:
#                 mesh, stats_dict = out
#             except TypeError:
#                 mesh, stats_dict = out, {}

#             mesh.export(os.path.join(out_dir, 'vis', '{}_{}_{}.off'.format(it, data_vis['category'], data_vis['it'])))


    # Save checkpoint
    if (checkpoint_every > 0 and (it % checkpoint_every) == 0):
        print('Saving checkpoint')
        checkpoint_io.save('metropolis+sigma1.pt', epoch_it=epoch_it, it=it)

    # Backup if necessary
    if (backup_every > 0 and (it % backup_every) == 0):
        print('Backup checkpoint')
        checkpoint_io.save('metropolis+sigma1Model_%d.pt' % it, epoch_it=epoch_it, it=it)
        
    # Run validation
#     if validate_every > 0 and (it % validate_every) == 0:
#         eval_dict = trainer.evaluate(val_loader)
#         metric_val = eval_dict[model_selection_metric]
#         print('Validation metric (%s): %.4f'
#                 % (model_selection_metric, metric_val))

#         for k, v in eval_dict.items():
#             logger.add_scalar('val/%s' % k, v, it)

#         if model_selection_sign * (metric_val - metric_val_best) > 0:
#             metric_val_best = metric_val
#             print('New best model (loss %.4f)' % metric_val_best)
#             checkpoint_io.save('model_best.pt', epoch_it=epoch_it, it=it,
#                                 loss_val_best=metric_val_best)

    # Exit if necessary
#     if exit_after > 0 and (time.time() - t0) >= exit_after:
#         print('Time limit reached. Exiting.')
#         checkpoint_io.save('model.pt', epoch_it=epoch_it, it=it,
#                             loss_val_best=metric_val_best)

#print(train_loader)

/home/nizar/miniconda3/envs/conv_onet/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[Epoch 100] it=100, loss=0.9952, time: 75.24s, 10:58
[Epoch 200] it=200, loss=-0.2721, time: 136.70s, 10:59
Saving checkpoint
[Epoch 300] it=300, loss=0.3416, time: 198.77s, 11:00
[Epoch 400] it=400, loss=0.2693, time: 262.06s, 11:01
Saving checkpoint
[Epoch 500] it=500, loss=0.4456, time: 325.89s, 11:02
[Epoch 600] it=600, loss=0.5506, time: 390.20s, 11:03
Saving checkpoint
[Epoch 700] it=700, loss=0.9491, time: 454.24s, 11:04
[Epoch 800] it=800, loss=0.9850, time: 517.54s, 11:05
Saving checkpoint
[Epoch 900] it=900, loss=0.5740, time: 580.04s, 11:06
[Epoch 1000] it=1000, loss=0.6208, time: 642.25s, 11:07
Saving checkpoint
[Epoch 1100] it=1100, loss=0.6259, time: 704.15s, 11:08
[Epoch 1200] it=1200, loss=0.8625, time: 766.26s, 11:09
Saving checkpoint
[Epoch 1300] it=1300, loss=1.0999, time: 828.68s, 11:10
[Epoch 1400] it=1400, loss=0.8184, time: 890.60s, 11:11
Saving checkpoint
[Epoch 1500] it=1500, loss=1.0128, time: 953.03s, 11:12
[Epoch 1600] it=1600, loss=0.8648, time: 1015.44s, 1

[Epoch 12500] it=12500, loss=0.5276, time: 7737.95s, 13:05
[Epoch 12600] it=12600, loss=0.5927, time: 7799.43s, 13:06
Saving checkpoint
[Epoch 12700] it=12700, loss=0.6644, time: 7861.04s, 13:07
[Epoch 12800] it=12800, loss=0.2762, time: 7922.51s, 13:09
Saving checkpoint
[Epoch 12900] it=12900, loss=0.5897, time: 7984.12s, 13:10
[Epoch 13000] it=13000, loss=0.5606, time: 8045.55s, 13:11
Saving checkpoint
[Epoch 13100] it=13100, loss=0.5699, time: 8106.83s, 13:12
[Epoch 13200] it=13200, loss=0.6254, time: 8168.20s, 13:13
Saving checkpoint
[Epoch 13300] it=13300, loss=0.6315, time: 8229.50s, 13:14
[Epoch 13400] it=13400, loss=0.6253, time: 8290.91s, 13:15
Saving checkpoint
[Epoch 13500] it=13500, loss=0.5627, time: 8352.39s, 13:16
[Epoch 13600] it=13600, loss=0.6806, time: 8413.83s, 13:17
Saving checkpoint
[Epoch 13700] it=13700, loss=0.6160, time: 8475.43s, 13:18
[Epoch 13800] it=13800, loss=0.5426, time: 8536.88s, 13:19
Saving checkpoint
[Epoch 13900] it=13900, loss=0.5299, time: 8599.

[Epoch 24400] it=24400, loss=0.5835, time: 15059.83s, 15:07
Saving checkpoint
[Epoch 24500] it=24500, loss=0.3140, time: 15121.70s, 15:08
[Epoch 24600] it=24600, loss=0.5503, time: 15183.66s, 15:10
Saving checkpoint
[Epoch 24700] it=24700, loss=-0.1906, time: 15245.65s, 15:11
[Epoch 24800] it=24800, loss=0.4754, time: 15307.24s, 15:12
Saving checkpoint
[Epoch 24900] it=24900, loss=0.3623, time: 15369.09s, 15:13
[Epoch 25000] it=25000, loss=-0.0596, time: 15430.86s, 15:14
Saving checkpoint
[Epoch 25100] it=25100, loss=0.2937, time: 15492.59s, 15:15
[Epoch 25200] it=25200, loss=0.4947, time: 15554.36s, 15:16
Saving checkpoint
[Epoch 25300] it=25300, loss=0.3812, time: 15616.24s, 15:17
[Epoch 25400] it=25400, loss=0.4960, time: 15678.00s, 15:18
Saving checkpoint
[Epoch 25500] it=25500, loss=0.5735, time: 15739.74s, 15:19
[Epoch 25600] it=25600, loss=0.4496, time: 15801.20s, 15:20
Saving checkpoint
[Epoch 25700] it=25700, loss=0.2215, time: 15862.58s, 15:21
[Epoch 25800] it=25800, loss=0.2